# Analysis Notebook for the scraper project

In [18]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import os
import sys
import re

In [19]:
# prevent hiding of columns
pd.options.display.max_columns = None

# prevent cropping of lng strings for display
pd.options.display.max_colwidth = 300

In [20]:
# no wrapping of long strings

In [21]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [86]:
root = Path(os.getcwd()).parent
fname = root / 'data' / 'room_database.csv'
df=pd.read_csv(fname)
df

,,,,,,,,,,,,Unnamed: 0
index_url,alt,class,data-actions,data-advert-position,data-attr-href,data-category,data-label,data-trace,fname,href,processed,scrape_ts
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",[],click,2042131_1220_cf282400a14da664718af86e91aa9d15,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,ContentList,ContentTitle,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_22-18-47_969143.scrape,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,False,2020-10-18 22:18:47.969143


In [47]:
df.set_index('Unnamed: 0')

,Unnamed: 0.1,href,fname,scrape_ts
Unnamed: 0,,,,
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,NaN,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_20-52-26_562935.scrape,2020-10-18 20:52:26.562935
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,NaN,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_20-53-21_484955.scrape,2020-10-18 20:53:21.484955
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,NaN,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_20-53-24_651930.scrape,2020-10-18 20:53:24.651930
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,NaN,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_20-53-27_798963.scrape,2020-10-18 20:53:27.798963
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-schoenem-haus.2073403?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,NaN,NaN,C:\coding\wgscraper\data\ronorp\adpage_2020-10-18_20-53-30_872928.scrape,2020-10-18 20:53:30.872928


In [27]:
a=df.to_dict()

In [66]:
a=pd.DataFrame()
a={'a':{'nr1':1,'nr2':2}}
a.update({'b':{'nr1':3,'nr2':4}})
b=pd.DataFrame(a).transpose()
b

,nr1,nr2
a,1,2
b,3,4


In [33]:
import datetime

In [43]:
datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S_%f")

'2020-10-18_16-53-42_257201'

In [69]:
from pathlib import Path
a={'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1': {'data-advert-position': '2042131_1220_20941f96bcb24379f597ac78d9fc0d78',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'alt': 'Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-50_909156.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 8, 50, 909156),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1': {'data-advert-position': '2073431_1220_995118c3e235ede9f70e283eaac961d1',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1',
  'alt': 'WG zimmer K5 befristet für 6 Monate (eventuell auch fuer...',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-55_236071.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 8, 55, 236071),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf': {'data-advert-position': '2073422_1220_ee0e5471d1e37033c2d82a73050ad0b0',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'alt': 'Mitbewohnerin gesucht ab 1. Nov/Dez od. nach Vereinbarung',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-58_837035.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 8, 58, 837035),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf': {'data-advert-position': '2073417_1220_4ace612802ebbbfe74b3f1383a14378a',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'alt': 'WG-Zimmer in 4er-WG in 5 min. Gehdistanz zum HB',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname': 'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-09-02_609034.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 9, 2, 609034),
  'processed': False},
 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf': {'data-advert-position': '2073410_1220_11278ecccec3fde69f6b169b9aa10632',
  'class': [],
  'href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'data-attr-href': 'https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf',
  'alt': 'Wunderschönes Zimmer mit eigenem Bad im 5. Stock in...',
  'data-trace': '',
  'data-category': 'ContentList',
  'data-label': 'ContentTitle',
  'data-actions': 'click',
  'fname':'C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-09-05_941036.scrape',
  'scrape_ts': datetime.datetime(2020, 10, 18, 22, 9, 5, 941036),
  'processed': False}}


In [82]:
b=pd.DataFrame(a).transpose()
b.index.name = 'link'
b

,data-advert-position,class,href,data-attr-href,alt,data-trace,data-category,data-label,data-actions,fname,scrape_ts,processed
link,,,,,,,,,,,,
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,2042131_1220_20941f96bcb24379f597ac78d9fc0d78,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,"Wunderschönes Co-Living Haus mit 38 Suiten in Kloten,...",,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-50_909156.scrape,2020-10-18 22:08:50.909156,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,2073431_1220_995118c3e235ede9f70e283eaac961d1,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1,WG zimmer K5 befristet für 6 Monate (eventuell auch fuer...,,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-55_236071.scrape,2020-10-18 22:08:55.236071,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2073422_1220_ee0e5471d1e37033c2d82a73050ad0b0,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/mitbewohnerin-gesucht-ab-1-nov-dez-od-nach-vereinbarung.2073422?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,Mitbewohnerin gesucht ab 1. Nov/Dez od. nach Vereinbarung,,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-08-58_837035.scrape,2020-10-18 22:08:58.837035,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2073417_1220_4ace612802ebbbfe74b3f1383a14378a,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-in-4er-wg-in-5-min-gehdistanz-zum-hb.2073417?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,WG-Zimmer in 4er-WG in 5 min. Gehdistanz zum HB,,ContentList,ContentTitle,click,C:/coding/wgscraper/data/ronorp/adpage_2020-10-18_22-09-02_609034.scrape,2020-10-18 22:09:02.609034,False
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.2073410?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1%26page%3D2%26hash%3D8066eefae06f4ebdde76511cbb811cdf,2073410_1220_11278ecccec3fde69f6b169b9aa10632,[],https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-zimmer-mit-eigenem-bad-im-5-stock-in-toplage.20734

In [84]:
b['href']

link
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1                                                                                                                          https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wunderschoenes-co-living-haus-mit-38-suiten-in-kloten.2042131?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1
https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1                                                                                                                    https://www.ronorp.net/zuerich/immobilien/wg-zuerich.1220/wg-zimmer-k5-befristet-fuer-6-monate-eventuell-auch-fuer.2073431?&back_url=%2Fzuerich%2Fimmobilien%2Fwg-zuerich.1220%3Fs%3D1
https://www.ronorp.net/zuerich/im